In [352]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
import numpy as np
import pandas as pd
import cvxpy as cp
import torch
from copy import copy, deepcopy


In [353]:
n=5

In [354]:
pen = 0.1
a=0.9
b=0.1
t= 0.3
t_ref = 0.6*np.ones(n)

In [355]:
np.random.rand()

0.009970348984377875

In [356]:
ds = []

In [357]:
for i in range(1000):
	pen = 0.3*np.random.rand() + 0.1
	a=0.4*np.random.rand() + 0.6
	b=0.2*np.random.rand() +0.1
	t= 0.4*np.random.rand() + 0.1
	t_ref = 0.4*np.array([np.random.rand()]*(n)) + 0.6

	Q = np.identity(n)
	L = pen * np.identity(n)
	I = np.identity(n)
	A = np.array([a**(i+1) for i in range(n)])
	B = np.array([[a**(i-1) * b for i in range(j, 0, -1)] + [0]*(n-j) for j in range(1, n+1)])

	x = cp.Variable(n)
	constraints = [x <= np.ones(n), -x <= np.zeros(n), (A*t +B@x) <= np.ones(n), -(A*t +B@x) <= np.zeros(n)]
	prob = cp.Problem(cp.Minimize(cp.quad_form(A*t +B@x - t_ref, I) + cp.quad_form(x, L)), constraints)
	prob.solve()
	if prob.status == 'optimal': ds.append((np.array([t, t_ref[0],a ,b, pen]), x.value))
	

In [358]:
prob.status

'optimal'

In [359]:
sol = torch.tensor(x.value)

In [360]:
lambda_ = torch.tensor(constraints[1].dual_value)

In [361]:
nu = torch.tensor(constraints[0].dual_value)

In [362]:
rho = torch.tensor(constraints[2].dual_value)

In [363]:
mu = torch.tensor(constraints[3].dual_value)

In [364]:
from pickle import dump
dump(ds, open("mpc.pkl", "wb"))

In [365]:
def grad_cost(U, x_t, x_ref, a, b, q, r):
	grad = torch.zeros_like(U)
	x = x_t
	for k in range(n):
		U_t = U[k].squeeze()
		x_next = a * x + b * U_t
		grad[k] = 2 * b * q * (x_next - x_ref[k].squeeze()) + 2 * r * U_t
		x = x_next
	return grad

In [366]:
def kkt_loss(U, lambda_, mu, nu, rho, x_t, x_ref, a, b, q, r):
        U_min = 0.0
        U_max = 1.0
        T_min = 0.0
        T_max = 1.0

        # Violazione della stazionarietà
        grad_J = grad_cost(U, x_t, x_ref, a, b, q, r)
        grad_ineq_control = lambda_ - mu
        grad_ineq_state = -b * nu + b * rho
        stationarity_violation = (grad_J + grad_ineq_control + grad_ineq_state).pow(2).sum()

        # Violazione della primal feasibility (controlli e stati)
        control_feasibility = (torch.max(U_min - U, torch.zeros_like(U)) + torch.max(U - U_max, torch.zeros_like(U))).pow(2).sum()

        x = copy(x_t)
        state_feasibility = 0
        for k in range(n):
            U_t = U[..., k].squeeze()
            x = copy(a * x + b * U_t)
            state_feasibility += (torch.max(T_min - x, torch.zeros_like(x)) + torch.max(x - T_max, torch.zeros_like(x))).pow(2).sum()

        # Violazione della complementarità
        comp_control = lambda_ * (U - U_min) + mu * (U_max - U)
        comp_state = 0
        x = copy(x_t)
        for k in range(n):
            U_t = U[k].squeeze()
            x_next = a * x + b * U_t
            nu_t = nu[k].squeeze()
            rho_t = rho[k].squeeze()
            comp_state += (nu_t * (T_min - x_next) + rho_t * (x_next - T_max)).pow(2).sum()
            x = copy(x_next)
        comp_loss = comp_control.pow(2).sum() + comp_state

        # Violazione della dual feasibility

        # Somma delle violazioni
        return grad_J, stationarity_violation, control_feasibility,state_feasibility,  comp_loss

In [367]:
kkt_loss(sol, lambda_, mu, nu, rho, t, t_ref, a, b, torch.tensor(1), pen)

(tensor([ 2.1016e-01,  1.8768e-01,  1.5076e-01,  9.2640e-02, -1.3264e-09],
        dtype=torch.float64),
 tensor(0.1106, dtype=torch.float64),
 tensor(0., dtype=torch.float64),
 tensor(0., dtype=torch.float64),
 tensor(2.6529e-07, dtype=torch.float64))

In [368]:
grad = torch.zeros_like(sol)
x = copy(t)
for k in range(n):
	U_t = copy(sol[k].squeeze())
	x_next = a * x + b * U_t
	grad[k] = 2 * b  * (x_next - t_ref[k].squeeze()) + 2 * pen * U_t- b*nu[k]
	x = copy(x_next)

In [369]:
lambda_

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

In [370]:
sol

tensor([1.0000, 0.9305, 0.8243, 0.6622, 0.4069], dtype=torch.float64)

In [371]:
grad

tensor([2.0998e-01, 1.8768e-01, 1.5076e-01, 9.2640e-02, 2.7756e-17],
       dtype=torch.float64)

In [372]:
pen

0.1646234299501639

In [373]:
sol[0]

tensor(1., dtype=torch.float64)

In [374]:
a * t + b * sol[0]

tensor(0.3019, dtype=torch.float64)

In [375]:
t_ref[0].squeeze()

0.8849300074796036